In [7]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
from splinter import Browser

In [8]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [9]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [10]:
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
response = requests.get(url)
soup = bs(response.text, 'lxml')

In [11]:
news_title = soup.find('div', class_= 'content_title').text
news_p = soup.find('div', class_= 'rollover_description_inner').text
print('-'*12)
print(news_title)
print(news_p)

------------


NASA to Broadcast Mars 2020 Perseverance Launch, Prelaunch Activities



Starting July 27, news activities will cover everything from mission engineering and science to returning samples from Mars to, of course, the launch itself.



In [12]:
img_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(img_url)

In [13]:
browser.click_link_by_partial_text('FULL IMAGE')
browser.click_link_by_partial_text('more info')
browser.click_link_by_partial_text('.jpg')

/Users/bhuffstetler/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/splinter/driver/webdriver/__init__.py:528: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


In [14]:

# Extra code added here to test
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

feat_img = soup.find('figure', class_='lede')

In [15]:
featured_image_url = browser.url
featured_image_url

'https://photojournal.jpl.nasa.gov/jpeg/PIA16217.jpg'

In [16]:
# Use pandas to extract the tables in the html
url = 'https://space-facts.com/mars/'
tables = pd.read_html(url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [17]:
type(tables)

list

In [18]:
# Creating a dataframe with the tables
df = tables[0]
df.columns = ['Description', 'Value']
df

,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [19]:
# Converting to HTML table string
html_table = df.to_html()
html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Description</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium 

In [22]:
!open table.html

In [25]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [26]:
# Get page html and make beautifulsoup object
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [27]:
# Get the html containing the titles and put into a list
title_list = soup.find_all('div', class_='description')

In [28]:
# Loop through the div objects and scrape titles and urls of hires images
# Initiate the list to store dictionaries
hemisphere_image_urls = []
for title in title_list:
    # Navigate browser to page then click on title link to hires image page
    browser.visit(url)
    browser.click_link_by_partial_text(title.a.h3.text)

    # Grab the destination page html and make into BeautifulSoup object
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    # Parse the hires image source(src) relative url then append to domain name
    # for absolute url 
    img_url_list = soup.find('img', class_='wide-image')
    img_url = f"https://astrogeology.usgs.gov{img_url_list['src']}"

    # Create dictionary with returned values and add dict to hemisphere_image_urls list
    post = {
            'title': title.a.h3.text,
            'image_url': img_url
            }
    hemisphere_image_urls.append(post)

/Users/bhuffstetler/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/splinter/driver/webdriver/__init__.py:528: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


In [29]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'image_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'image_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'image_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'image_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]